## Import các thư viện cần thiết

In [20]:
import pandas as pd
import numpy as np

## Khám phá dữ liệu

### Đọc dữ liệu từ file

In [21]:
weather_df = None
weather_df = pd.read_csv('../data/internal/raw/weather.csv')

In [22]:
weather_df

,Country,Name,Weather,Temp,Humidity,Visibility,Wind speed,Clouds
0,JP,Tokyo,Clear,280.77,57,10000,3.60,0
1,ID,Pecenongan,Haze,298.99,83,6000,2.06,20
2,IN,New Delhi,Mist,293.25,83,1500,0.00,0
3,CN,Guangzhou,Clouds,288.07,47,10000,5.16,27
4,IN,Konkan Division,Mist,300.18,83,2200,3.09,20
...,...,...,...,...,...,...,...,...
3503,JP,Toyokawa,Clouds,280.24,50,10000,2.68,19
3504,US,City of Newport News,Clear,285.26,56,10000,4.12,0
3505,SS,Yei,Clouds,297.75,66,10000,1.02,56
3506,IN,Adoni,Clear,297.39,70,10000,3.74,6


### Chuyển đổi định dạng ISO2 sang tên đầy đủ

In [23]:
world_cities = pd.read_csv("../data/external/worldcities.csv")


# Tạo mapping từ iso2 sang tên đầy đủ
iso2_to_country = dict(zip(world_cities['iso2'], world_cities['country']))

# Thay thế giá trị trong cột 'Country' từ iso2 sang tên đầy đủ
weather_df['Country'] = weather_df['Country'].replace(iso2_to_country)
weather_df

,Country,Name,Weather,Temp,Humidity,Visibility,Wind speed,Clouds
0,Japan,Tokyo,Clear,280.77,57,10000,3.60,0
1,Indonesia,Pecenongan,Haze,298.99,83,6000,2.06,20
2,India,New Delhi,Mist,293.25,83,1500,0.00,0
3,China,Guangzhou,Clouds,288.07,47,10000,5.16,27
4,India,Konkan Division,Mist,300.18,83,2200,3.09,20
...,...,...,...,...,...,...,...,...
3503,Japan,Toyokawa,Clouds,280.24,50,10000,2.68,19
3504,United States,City of Newport News,Clear,285.26,56,10000,4.12,0
3505,South Sudan,Yei,Clouds,297.75,66,10000,1.02,56
3506,India,Adoni,Clear,297.39,70,10000,3.74,6


### Dữ liệu có bao nhiêu cột và bao nhiêu dòng?

In [24]:
shape = None 
shape = weather_df.shape
print(shape)

(3508, 8)


### Ý nghĩa của mỗi dòng là gì?


Mỗi dòng chứa thông tin về tình hình thời tiết của một thành phố.

### Dữ liệu có dòng nào bị trùng lặp không?

In [25]:
num_duplicated_rows = None
num_duplicated_rows = weather_df.duplicated().sum()

In [26]:
#TEST
if num_duplicated_rows == 0:
    print(f"Dữ liệu không có dòng nào trùng lặp!")
else:
    print(f"Dữ liệu có {num_duplicated_rows} dòng bị trùng lặp "+ ". Vui lòng xóa các dòng đó!")

Dữ liệu có 6 dòng bị trùng lặp . Vui lòng xóa các dòng đó!


#### Xóa các dòng bị trùng lắp dữ liệu

In [27]:
#De-deduplicate the raw data
weather_df = weather_df.drop_duplicates()

#### Kiểm tra lại

In [28]:
#TEST
num_duplicated_rows = weather_df.duplicated().sum()
if num_duplicated_rows == 0:
    print(f"Dữ liệu không có dòng nào trùng lặp!")
else:
    print(f"Dữ liệu có {num_duplicated_rows} dòng bị trùng lặp "+ ". Vui lòng xóa các dòng đó!")

Dữ liệu không có dòng nào trùng lặp!


### Ý nghĩa của mỗi cột là gì?

- Country: ID của quốc gia
- Name: Tên của thành phố
- Weather: Điều kiện thời tiết
- Temp: Nhiệt độ hiện tại
- Humidity: Độ ẩm hiện tại
- Visibility: Tầm nhìn hiện tại
- Wind speed: Tốc độ gió hiện tại
- Clouds: Tần suất mây hiện tại

### Mỗi cột có kiểu dữ liệu gì? Có cột nào có kiểu dữ liệu không phù hợp cho việc xử lí dữ liệu không?

In [29]:

dtypes = weather_df.dtypes

In [30]:
dtypes

Country        object
Name           object
Weather        object
Temp          float64
Humidity        int64
Visibility      int64
Wind speed    float64
Clouds          int64
dtype: object

Không có cột nào có kiểu dữ liệu không phù hợp cho việc xử lí dữ liệu!

### Giá trị của các cột có kiểu numeric phân bố như thế nào?
- Phần trăm các giá trị bị thiếu là bao nhiêu?
- Giá trị lớn nhất? Giá trị nhỏ nhất? Có gì bất thường không?

Đối với các cột có kiểu numeric, chúng ta sẽ tính:
- Phần trăm (từ 0 tới 100) của giá trị bi thiếu
- Giá trị nhỏ nhất
- Tứ phân vị 25
- Tứ phân vị 50
- Tứ phân vị 75
- Giá trị lớn nhất

Lưu kết quả vào DataFrame `num_col_info_df`, trong đó:
- Tên của các cột lả tên của các cột có kiểu numeric trong `df`
- Tên của các dòng: "missing_ratio", "min", "lower_quartile", "median", "upper_quartile", "max"

In [31]:
num_col_info_df = None

num_col_info_df = weather_df.select_dtypes(include=np.number).copy()

def missing_ratio(column):
    return ((column.isnull().sum() / column.shape[0]) * 100).round(1)

def lower_quartile(column):
    return (column.quantile(0.25)).round(1)

def median(column):
    return (column.median())

def upper_quartile(column):
    return (column.quantile(0.75)).round(1)

# Làm tròn giá trị đến 1 chữ số thập phân
num_col_info_df = num_col_info_df.round(1)

num_col_info_df = num_col_info_df.agg([missing_ratio, "min", lower_quartile, median, upper_quartile, "max"])

In [32]:
num_col_info_df

,Temp,Humidity,Visibility,Wind speed,Clouds
missing_ratio,0.0,0.0,0.0,0.0,0.0
min,247.6,1.0,38.0,0.0,0.0
lower_quartile,278.3,50.0,10000.0,1.5,5.0
median,289.3,69.0,10000.0,2.5,40.0
upper_quartile,298.2,84.0,10000.0,3.7,93.0
max,313.8,100.0,10000.0,16.1,100.0


### Xóa cột có tỉ lệ giá trị bị thiếu lớn hơn 75%

In [33]:
def drop_missing_features(df: pd.DataFrame, missing_lst: dict = dict(num_col_info_df.iloc[0]), threshold: float = 75.0) -> pd.DataFrame:
    for col, missing_rate in missing_lst.items():
        if missing_rate > threshold:
            del df[col]
    return df

In [34]:
weather_df = drop_missing_features(weather_df)

In [35]:
weather_df.head()

,Country,Name,Weather,Temp,Humidity,Visibility,Wind speed,Clouds
0,Japan,Tokyo,Clear,280.77,57,10000,3.60,0
1,Indonesia,Pecenongan,Haze,298.99,83,6000,2.06,20
2,India,New Delhi,Mist,293.25,83,1500,0.00,0
3,China,Guangzhou,Clouds,288.07,47,10000,5.16,27
4,India,Konkan Division,Mist,300.18,83,2200,3.09,20


### Giá trị của các cột có kiểu không phải numeric phân bố như thế nào?
- Phần trăm giá trị bị thiếu là bao nhiêu?
- Có bao nhiêu giá trị phân biệt? Hiển thị một vài giá trị
- Có gì bất thường không?

Đối với các cột có kiểu numeric, chúng ta sẽ tính:
- Phần trăm (từ 0 tới 100) của giá trị bị thiếu
- Số lượng giá trị (giá trị phân biệt, không xét các giá trị bị thiếu)
Lưu kết quả vào DataFrame num_col_info_df, trong đó:
- Tên của các cột lả tên của các cột có kiểu không phải numeric trong `df`
- Tên của các dòng: "missing_ratio", "num_values"

In [36]:
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_columns', None)

In [37]:
cat_col_info_df = weather_df.select_dtypes(exclude=[np.number])

def missing_ratio(column):
    return ((column.isnull().sum() / column.shape[0]) * 100).round(1)

# Hàm tính số lượng giá trị
def num_values(column):
    return column.nunique()

# Hàm tính tỷ lệ của từng giá trị
def value_ratios(column):
    value_counts = column.value_counts() #Đếm số lượng của mỗi loại value trong 1 cột
    non_missing_count = value_counts.sum() #Tổng số lượng của tất cả value trong 1 cột
    ratios = (value_counts / non_missing_count * 100).round(1) #Lưu tỉ lệ vào Series
    ratios_dict = ratios.to_dict()
    sorted_ratios_dict = dict(sorted(ratios_dict.items(), key=lambda item: item[1], reverse=True))
    return sorted_ratios_dict

cat_col_info_df = cat_col_info_df.agg([missing_ratio, num_values, value_ratios])
cat_col_info_df

,Country,Name,Weather
missing_ratio,0.0,0.1,0.0
num_values,238,3336,10
value_ratios,"{'China': 17.9, 'India': 8.3, 'United States': 7.4, 'Japan': 4.3, 'Brazil': 4.3, 'Russia': 2.9, ...","{'Adana': 0.1, 'Xinzhou': 0.1, 'George Town': 0.1, 'Xinxiang': 0.1, 'Guadalupe': 0.1, 'Minamisen...","{'Clouds': 58.0, 'Clear': 28.5, 'Rain': 6.4, 'Mist': 3.1, 'Snow': 1.8, 'Haze': 1.3, 'Fog': 0.4, ..."


### Lưu dữ liệu đã xử lí

In [38]:
print(f"Total number of features: {weather_df.shape[1]}")
weather_df.dtypes

Total number of features: 8


Country        object
Name           object
Weather        object
Temp          float64
Humidity        int64
Visibility      int64
Wind speed    float64
Clouds          int64
dtype: object

In [39]:

weather_df.to_csv("../data/internal/processed/weather_processed.csv" , index=False)